In [10]:
from datetime import datetime
from datetime import timezone


start_date = datetime(2023, 8, 8, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 8, 9, 23, 59, 59).replace(tzinfo=timezone.utc)

In [11]:
import polars as pl
import os
import json

data = {"date": [],
        "system name": [],
        "variant": [],
        "CO2_raw": [],
        "CO2_compensated": [],
        "CO2_filtered": [],
        "sht45_humidity": [],
        "bme280_humidity": [],
        "bme280_pressure": [],
        "sht45_temperature": [],
        "bme280_temperature": [],
        "chamber_temperature": [],
        "gas_bottle_id": []}

In [12]:
df = pl.read_csv("../data/averaged_gases.csv")
df.head()

Bottle_ID,CO2_dry
i64,f64
1,798.115438
2,807.238809
3,391.345476
4,399.530674
5,416.594961


In [13]:
quickflow_directory = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Software/quickflow/notebooks/acropolis/"
df_quickflow = pl.scan_parquet(os.path.join(quickflow_directory, "measurements.parquet")) 

In [15]:
# loop through the quickflow dataset and sort it into dataframe
df_q = df_quickflow.select(pl.col("creation_timestamp","sensor_name","value")).sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()

for payload in df_q.iter_rows():
    creation_timestamp = payload[0]
    sensor_name = payload[1]
    entry = json.loads(payload[2])
    
    if entry["variant"] == "calibration":
        # base
        data["date"].append(creation_timestamp)
        data["system name"].append(sensor_name)
        data["variant"].append(entry["variant"])
        #variant measurement & calibrtion
        data["CO2_raw"].append(entry["data"]["raw"])
        data["CO2_compensated"].append(entry["data"]["compensated"])
        data["CO2_filtered"].append(entry["data"]["compensated"])
        data["sht45_humidity"].append(entry["data"]["sht45_humidity"])
        data["bme280_humidity"].append(entry["data"]["bme280_humidity"])
        data["bme280_pressure"].append(entry["data"]["bme280_pressure"])
        data["sht45_temperature"].append(entry["data"]["sht45_temperature"])
        data["bme280_temperature"].append(entry["data"]["bme280_temperature"])
        data["chamber_temperature"].append(entry["data"]["chamber_temperature"])
        if entry["variant"] == "calibration":
                data["gas_bottle_id"].append(entry["data"]["gas_bottle_id"])
        else:
                data["gas_bottle_id"].append(None)
        
df = pl.DataFrame(data)
df.head()    

date,system name,variant,CO2_raw,CO2_compensated,CO2_filtered,sht45_humidity,bme280_humidity,bme280_pressure,sht45_temperature,bme280_temperature,chamber_temperature,gas_bottle_id
"datetime[μs, UTC]",str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2023-08-08 09:08:51.550 UTC,"""tum-esm-midcos…","""calibration""",0.0,0.0,0.0,45.49,44.92,940.85,19.4,18.76,18.8,"""40"""
2023-08-08 09:09:01.570 UTC,"""tum-esm-midcos…","""calibration""",417.9,446.4,446.4,44.3,42.01,950.21,19.4,18.77,18.8,"""40"""
2023-08-08 09:09:11.580 UTC,"""tum-esm-midcos…","""calibration""",500.2,528.9,528.9,39.5,36.77,950.9,19.4,18.77,18.8,"""40"""
2023-08-08 09:09:21.590 UTC,"""tum-esm-midcos…","""calibration""",531.1,561.6,561.6,37.21,34.11,950.04,19.42,18.79,18.8,"""40"""
2023-08-08 09:09:31.590 UTC,"""tum-esm-midcos…","""calibration""",545.2,577.5,577.5,35.99,32.57,951.24,19.43,18.78,18.8,"""40"""
